# Prática CRUD com SQLite3 em Python

In [ ]:
# Importa o módulo da BIBLIOTECA PADRÃO do Python para trabalhar com bancos SQLite.
# Não precisa instalar com pip. Com ele você cria/abre um arquivo .db,
# executa SQL (CREATE/INSERT/SELECT/UPDATE/DELETE) e faz commit/close.
import sqlite3 as sq

## 1. Criando conexão com o banco SQLite

In [ ]:
# Abre (ou cria, se não existir) o arquivo de banco de dados "usuarios.db"
# O arquivo será salvo no diretório atual do notebook/script.
# A variável 'con' é o objeto de CONEXÃO: controla transações (commit/rollback) e o acesso ao BD.
con = sqlite3.connect("usuarios.db")

# A partir da conexão, criamos um CURSOR.
# O cursor é quem envia comandos SQL (CREATE TABLE, INSERT, SELECT, UPDATE, DELETE) ao banco.
cur = con.cursor()

## 2. Criando tabelas

In [ ]:
# ---- DDL: criação das tabelas (idempotente) ----
cur.executescript("""
CREATE TABLE IF NOT EXISTS usuarios (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    senha TEXT NOT NULL,
    data_criacao TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS produtos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    preco REAL NOT NULL,
    estoque INTEGER NOT NULL
);

CREATE TABLE IF NOT EXISTS pedidos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    usuario_id INTEGER NOT NULL,
    data_pedido TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    status TEXT DEFAULT 'Pendente',
    FOREIGN KEY (usuario_id) REFERENCES usuarios(id)
);

CREATE TABLE IF NOT EXISTS itens_pedido (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pedido_id INTEGER NOT NULL,
    produto_id INTEGER NOT NULL,
    quantidade INTEGER NOT NULL,
    preco_unitario REAL NOT NULL,
    FOREIGN KEY (pedido_id) REFERENCES pedidos(id),
    FOREIGN KEY (produto_id) REFERENCES produtos(id)
);
""")
con.commit()

# ---- Índice único para produtos.nome (para UPSERT por nome) ----
try:
    cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS idx_produtos_nome ON produtos(nome);")
    con.commit()
except Exception:
    # Se já existirem duplicatas, remove mantendo a primeira
    cur.execute("""
        DELETE FROM produtos
        WHERE rowid NOT IN (
            SELECT MIN(rowid) FROM produtos GROUP BY nome
        );
    """)
    con.commit()
    cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS idx_produtos_nome ON produtos(nome);")
    con.commit()

# ---- SEED: dados base ----
usuarios = [
    ("João Silva",  "joao@email.com",   "123456"),
    ("Maria Souza", "maria@email.com",  "abcdef"),
    ("Carlos Pereira", "carlos@email.com", "senha123"),
]
produtos = [
    ("Camiseta",        29.99, 100),
    ("Calça Jeans",     79.99,  50),
    ("Tênis Esportivo",129.99,  30),
]

# ---- UPSERT: insere ou atualiza se já existir (idempotente) ----
cur.executemany("""
    INSERT INTO usuarios (nome, email, senha)
    VALUES (?, ?, ?)
    ON CONFLICT(email) DO UPDATE SET
        nome  = excluded.nome,
        senha = excluded.senha;
""", usuarios)

cur.executemany("""
    INSERT INTO produtos (nome, preco, estoque)
    VALUES (?, ?, ?)
    ON CONFLICT(nome) DO UPDATE SET
        preco   = excluded.preco,
        estoque = excluded.estoque;
""", produtos)
con.commit()

# ---- READ: listar dados ----
print("Usuários:")
for row in cur.execute("SELECT id, nome, email, datetime(data_criacao) FROM usuarios ORDER BY id"):
    print(row)

print("\nProdutos:")
for row in cur.execute("SELECT id, nome, preco, estoque FROM produtos ORDER BY id"):
    print(row)

# ---- UPDATE seguro: baixa 1 do estoque do produto id=1 se houver estoque ----
qtd = 1
produto_id = 1
cur.execute(
    "UPDATE produtos SET estoque = estoque - ? WHERE id = ? AND estoque >= ?",
    (qtd, produto_id, qtd)
)
if cur.rowcount == 0:
    print("\n[UPDATE] Sem estoque suficiente ou produto inexistente para id=1.")
else:
    con.commit()
    print("\n[UPDATE] Estoque do produto id=1 atualizado com sucesso.")

# Mostrar produtos após o UPDATE
print("\nProdutos (após UPDATE):")
for row in cur.execute("SELECT id, nome, preco, estoque FROM produtos ORDER BY id"):
    print(row)

# ---- DELETE: tentar excluir usuário id=3 (vai falhar se houver FKs) ----
usuario_id = 3
try:
    cur.execute("DELETE FROM usuarios WHERE id = ?", (usuario_id,))
    if cur.rowcount == 0:
        print(f"\n[DELETE] Nenhum usuário com id={usuario_id}.")
    else:
        con.commit()
        print(f"\n[DELETE] Usuário id={usuario_id} excluído com sucesso.")
except sqlite3.IntegrityError as e:
    print("\n[DELETE] Exclusão bloqueada por vínculo (FK):", e)

# ---- Métricas rápidas ----
media_preco = cur.execute("SELECT AVG(preco) FROM produtos").fetchone()[0]
total_prod  = cur.execute("SELECT COUNT(*) FROM produtos").fetchone()[0]
print(f"\nPreço médio dos produtos: {media_preco:.2f}" if media_preco is not None else "\nPreço médio dos produtos: sem dados")
print(f"Total de produtos cadastrados: {total_prod}")

# ---- Encerrar conexão ----
con.close()

## 3. Inserindo dados (CREATE)

In [ ]:
# ==========================
# Seed idempotente (uma célula)
# ==========================
import sqlite3

# Use 'with' para garantir que a conexão fica aberta durante todo o bloco
# e que o commit/close acontecem corretamente ao final.
with sqlite3.connect("usuarios.db") as con:
    cur = con.cursor()

    # Ativa validação de chaves estrangeiras nesta conexão
    cur.execute("PRAGMA foreign_keys = ON;")

    # ---- Cria tabelas se não existirem ----
    cur.executescript("""
    CREATE TABLE IF NOT EXISTS usuarios (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome  TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        senha TEXT NOT NULL,
        data_criacao TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome   TEXT NOT NULL,
        preco  REAL NOT NULL,
        estoque INTEGER NOT NULL
    );

    CREATE TABLE IF NOT EXISTS pedidos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        usuario_id INTEGER NOT NULL,
        data_pedido TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        status TEXT DEFAULT 'Pendente',
        FOREIGN KEY (usuario_id) REFERENCES usuarios(id)
    );

    CREATE TABLE IF NOT EXISTS itens_pedido (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        pedido_id  INTEGER NOT NULL,
        produto_id INTEGER NOT NULL,
        quantidade INTEGER NOT NULL,
        preco_unitario REAL NOT NULL,
        FOREIGN KEY (pedido_id)  REFERENCES pedidos(id),
        FOREIGN KEY (produto_id) REFERENCES produtos(id)
    );
    """)

    # Índice único para permitir UPSERT por nome em 'produtos'
    cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS idx_produtos_nome ON produtos(nome);")

    # ---- Dados para inserir/atualizar ----
    usuarios = [
        ("João Silva",   "joao@email.com",   "123456"),
        ("Maria Souza",  "maria@email.com",  "abcdef"),
        ("Carlos Pereira","carlos@email.com","senha123"),
    ]
    produtos = [
        ("Camiseta",        29.99, 100),
        ("Calça Jeans",     79.99,  50),
        ("Tênis Esportivo",129.99,  30),
    ]

    # ---- UPSERT: insere ou atualiza se já existir ----
    cur.executemany("""
        INSERT INTO usuarios (nome, email, senha)
        VALUES (?, ?, ?)
        ON CONFLICT(email) DO UPDATE SET
            nome  = excluded.nome,
            senha = excluded.senha;
    """, usuarios)

    cur.executemany("""
        INSERT INTO produtos (nome, preco, estoque)
        VALUES (?, ?, ?)
        ON CONFLICT(nome) DO UPDATE SET
            preco   = excluded.preco,
            estoque = excluded.estoque;
    """, produtos)

    # Commit explícito (também será feito automaticamente ao sair do 'with')
    con.commit()

    # ---- Conferência ----
    print("Usuários:")
    for row in cur.execute("SELECT id, nome, email, datetime(data_criacao) FROM usuarios ORDER BY id"):
        print(row)

    print("\nProdutos:")
    for row in cur.execute("SELECT id, nome, preco, estoque FROM produtos ORDER BY id"):
        print(row)

# Obs.: ao sair do bloco 'with', a conexão é fechada.
# Se for executar MAIS comandos depois, abra outra conexão:
# con = sqlite3.connect("usuarios.db"); cur = con.cursor()

## 4. Consultando dados (READ)

In [ ]:
# Consultando usuários
print("Usuários:")  # título para a listagem de usuários no console

# Executa a query e itera diretamente sobre o cursor.
# Cada 'row' é uma tupla no formato: (id, nome, email, senha, data_criacao)
for row in cur.execute("SELECT * FROM usuarios"):
    print(row)  # imprime a tupla inteira

# Consultando produtos
print("\nProdutos:")  # \n adiciona uma linha em branco antes do próximo título

# Executa a query e percorre os resultados da tabela 'produtos'.
# Cada 'row' é uma tupla: (id, nome, preco, estoque)
for row in cur.execute("SELECT * FROM produtos"):
    print(row)  # imprime a tupla inteira

## 5. Atualizando dados (UPDATE)

In [ ]:
# Diminui em 1 a quantidade em estoque do produto com id = 1
# Atenção: este UPDATE permite estoque negativo se já estiver em 0
cur.execute("UPDATE produtos SET estoque = estoque - 1 WHERE id = 1")

# Persiste a alteração no arquivo .db
con.commit()

# Consulta para conferir o resultado após a atualização
print("\nEstoque atualizado:")
for row in cur.execute("SELECT * FROM produtos"):
    print(row)  # (id, nome, preco, estoque)

## 6. Deletando dados (DELETE)

In [ ]:
# Exclui o usuário de id = 3 (se existir).
# ATENÇÃO: se existir um pedido vinculado a esse usuário e o PRAGMA foreign_keys estiver ON,
# o SQLite vai bloquear a exclusão (violação de chave estrangeira).
cur.execute("DELETE FROM usuarios WHERE id = 3")

# Confirma a transação (torna a exclusão permanente no arquivo .db)
con.commit()

# Confere como ficou a tabela após a exclusão
print("\nUsuários após exclusão:")
for row in cur.execute("SELECT * FROM usuarios"):
    print(row)

## 7. Consultas com agregação

In [ ]:
# ----- Versão básica (mesma lógica que você usou) -----

print("\nPreço médio dos produtos:")
# Executa a agregação AVG(preco). O cursor retorna linhas; cada 'row' é uma tupla.
# Aqui virá só 1 linha, com 1 coluna (a média).
for row in cur.execute("SELECT AVG(preco) FROM produtos"):
    print(row)  # imprime a tupla, ex.: (79.99,)

print("\nTotal de produtos cadastrados:")
# COUNT(*) conta todas as linhas da tabela 'produtos'
for row in cur.execute("SELECT COUNT(*) FROM produtos"):
    print(row)  # imprime a tupla, ex.: (3,)

## 8. Exercício: Criando e manipulando uma tabela de categorias

Agora, crie uma tabela chamada categorias para armazenar as categorias dos produtos.
Em seguida:

1. Insira pelo menos 3 registros.

2. Liste todas as categorias.

3. Atualize o nome de uma categoria.

4. Exclua uma categoria.

In [ ]:
import sqlite3
import pandas as pd

# ---------------------------
# Conectar (usa o mesmo banco cinema.db ou um novo)
# ---------------------------
conn = sqlite3.connect("cinema.db")
cur = conn.cursor()

# ---------------------------
# 1. Criar tabela categorias
# ---------------------------
cur.executescript("""
DROP TABLE IF EXISTS categorias;

CREATE TABLE categorias (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL
);
""")
conn.commit()

# ---------------------------
# 2. Inserir pelo menos 3 registros
# ---------------------------
cur.executemany("""
INSERT INTO categorias (nome) VALUES (?);
""", [("Eletrônicos",), ("Roupas",), ("Alimentos",)])
conn.commit()

# ---------------------------
# 3. Listar todas as categorias
# ---------------------------
df1 = pd.read_sql_query("SELECT * FROM categorias;", conn)
print("Categorias cadastradas:")
display(df1)

# ---------------------------
# 4. Atualizar o nome de uma categoria
# Exemplo: mudar 'Roupas' para 'Moda'
# ---------------------------
cur.execute("""
UPDATE categorias
SET nome = 'Moda'
WHERE nome = 'Roupas';
""")
conn.commit()

df2 = pd.read_sql_query("SELECT * FROM categorias;", conn)
print("Após atualização:")
display(df2)

# ---------------------------
# 5. Excluir uma categoria
# Exemplo: apagar 'Alimentos'
# ---------------------------
cur.execute("""
DELETE FROM categorias
WHERE nome = 'Alimentos';
""")
conn.commit()

df3 = pd.read_sql_query("SELECT * FROM categorias;", conn)
print("Após exclusão:")
display(df3)

# Fechar conexão
conn.close()
